In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

In [2]:
PATH = Path().parent
DDIR = PATH / 'datasets'

In [3]:
stem_factory = StemmerFactory()
stemmer = stem_factory.create_stemmer()

stop_factory = StopWordRemoverFactory()
stop_word = stop_factory.get_stop_words()

In [6]:
stop_word

['yang',
 'untuk',
 'pada',
 'ke',
 'para',
 'namun',
 'menurut',
 'antara',
 'dia',
 'dua',
 'ia',
 'seperti',
 'jika',
 'jika',
 'sehingga',
 'kembali',
 'dan',
 'tidak',
 'ini',
 'karena',
 'kepada',
 'oleh',
 'saat',
 'harus',
 'sementara',
 'setelah',
 'belum',
 'kami',
 'sekitar',
 'bagi',
 'serta',
 'di',
 'dari',
 'telah',
 'sebagai',
 'masih',
 'hal',
 'ketika',
 'adalah',
 'itu',
 'dalam',
 'bisa',
 'bahwa',
 'atau',
 'hanya',
 'kita',
 'dengan',
 'akan',
 'juga',
 'ada',
 'mereka',
 'sudah',
 'saya',
 'terhadap',
 'secara',
 'agar',
 'lain',
 'anda',
 'begitu',
 'mengapa',
 'kenapa',
 'yaitu',
 'yakni',
 'daripada',
 'itulah',
 'lagi',
 'maka',
 'tentang',
 'demi',
 'dimana',
 'kemana',
 'pula',
 'sambil',
 'sebelum',
 'sesudah',
 'supaya',
 'guna',
 'kah',
 'pun',
 'sampai',
 'sedangkan',
 'selagi',
 'sementara',
 'tetapi',
 'apakah',
 'kecuali',
 'sebab',
 'selain',
 'seolah',
 'seraya',
 'seterusnya',
 'tanpa',
 'agak',
 'boleh',
 'dapat',
 'dsb',
 'dst',
 'dll',
 'dahulu

In [4]:
df = pd.read_excel(f'{DDIR}/Dataset B.xlsx')

In [6]:
df.sample(5)

,No,Reviews,Label
34,35,Baguss tapi ada yg rusak mohon untuk segera di...,0
147,148,"gila, pesan berapa di kirim berapa. ngajuin pe...",0
98,99,"Bagus, tapi strapnya kok beda ya warna item sa...",0
173,174,Pengaturanya patah jd susah mw ngatur jam nya,0
116,117,Kecewaaaa banget kok datengnya yang lain sih,0


In [7]:
df.set_index('No')

,Reviews,Label
No,,
1,Pesen 7 yang dikirim 1 doang.. respon penjual ...,0
2,kacamata nya ngga ada padahal pembayaran full ...,0
3,"Real pict,,,cumannn pengiriman agak lama sihhh...",0
4,Bagus sukakkkkkk pengirimannya cepet cuma Dari...,1
5,Pengemasan tokoh lama sekali melebihi 3hri bar...,0
...,...,...
196,"Paket sudah sampai.. baguss, bltidak ada yang ...",1
197,"Udah langganan sama tokonya,dan ngga pernah me...",1
198,Ke-2 kali beli tas disini dan tdk pernah menge...,1
